In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import T5Config, TFT5Model, T5Model, load_tf_weights_in_t5, T5Tokenizer
from transformers import T5ForConditionalGeneration, TFT5ForConditionalGeneration

/home/ubuntu/.local/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
!rm -rf temp2

In [4]:
import os

out = 'temp2'
os.makedirs(out, exist_ok=True)

In [5]:
config = T5Config.from_pretrained('malay-huggingface/t5-tiny-bahasa-cased')
config.save_pretrained(out)

In [6]:
!ls t5-tiny-noisy-en-ms

checkpoint
events.out.tfevents.1659110853.huseincomel-desktop
graph.pbtxt
model.ckpt-1024000.data-00000-of-00002
model.ckpt-1024000.data-00001-of-00002
model.ckpt-1024000.index
model.ckpt-1024000.meta
model.ckpt-1048000.data-00000-of-00002
model.ckpt-1048000.data-00001-of-00002
model.ckpt-1048000.index
model.ckpt-1048000.meta
model.ckpt-1072000.data-00000-of-00002
model.ckpt-1072000.data-00001-of-00002
model.ckpt-1072000.index
model.ckpt-1072000.meta
model.ckpt-1095000.data-00000-of-00002
model.ckpt-1095000.data-00001-of-00002
model.ckpt-1095000.index
model.ckpt-1095000.meta
model.ckpt-1119000.data-00000-of-00002
model.ckpt-1119000.data-00001-of-00002
model.ckpt-1119000.index
model.ckpt-1119000.meta
model.ckpt-1143000.data-00000-of-00002
model.ckpt-1143000.data-00001-of-00002
model.ckpt-1143000.index
model.ckpt-1143000.meta
model.ckpt-1166000.data-00000-of-00002
model.ckpt-1166000.data-00001-of-00002
model.ckpt-1166000.index
model.ckpt-1166000.meta
model.ckpt-1190000.data-00000-of-0000

In [7]:
model = T5Model(config)
load_tf_weights_in_t5(model, config, 't5-tiny-noisy-en-ms/model.ckpt-1302000')

T5Model(
  (shared): Embedding(32128, 384)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 384)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=384, out_features=768, bias=False)
              (k): Linear(in_features=384, out_features=768, bias=False)
              (v): Linear(in_features=384, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=384, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): FusedRMSNorm(torch.Size([384]), eps=1e-06, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=384, out_features=1344, bias=False)
              (wo): Linear(in_features=1344, out_features=384, bias=

In [8]:
tokenizer = T5Tokenizer('sp10m.cased.ms-en.model', padding = True)

In [9]:
model.save_pretrained(out)

In [10]:
model_gen = T5ForConditionalGeneration.from_pretrained(out)

In [11]:
model_tf = TFT5ForConditionalGeneration.from_pretrained(out, from_pt = True)

2022-08-01 10:53:01.824014: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 10:53:01.827858: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 10:53:01.828616: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 10:53:01.829507: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [12]:
with open('en-ms-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('en-ms-left.test') as fopen:
    left = fopen.read().split('\n')

In [13]:
model_gen.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 384)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 384)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=384, out_features=768, bias=False)
              (k): Linear(in_features=384, out_features=768, bias=False)
              (v): Linear(in_features=384, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=384, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): FusedRMSNorm(torch.Size([384]), eps=1e-06, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=384, out_features=1344, bias=False)
              (wo): Linear(in_features=1344, out_

In [14]:
from tqdm import tqdm

batch_size = 1

results = []
for i in tqdm(range(0, len(left), batch_size)):
    input_ids = [{'input_ids': tokenizer.encode(f'terjemah Inggeris ke Melayu: {s}', return_tensors = 'pt')[0]} for s in left[i:i + batch_size]]
    padded = tokenizer.pad(input_ids, padding = 'longest')
    outputs = model_gen.generate(padded['input_ids'].cuda(), attention_mask = padded['attention_mask'].cuda(), max_length = 1000)
    for o in outputs:
        results.append(tokenizer.decode(o, skip_special_tokens=True))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 69744/69744 [3:49:10<00:00,  5.07it/s]


In [15]:
# from tqdm import tqdm

# batch_size = 2

# results = []
# for i in tqdm(range(0, len(left[:1]), batch_size)):
#     input_ids = [{'input_ids': tokenizer.encode(f'terjemah Inggeris ke Melayu: {s}', return_tensors = 'pt')[0]} for s in left[i:i + batch_size]]
#     padded = tokenizer.pad(input_ids, padding = 'longest')
#     outputs = model_gen.generate(**padded, max_length = 1000)
#     for o in outputs:
#         results.append(tokenizer.decode(o, skip_special_tokens=True))

In [16]:
results[0]

'Anda tahu bagaimana kisah itu berlaku. Dua orang yang heteroseksual, hampir pasti orang kulit putih berjuang dalam hidup mereka. Mereka bertemu satu sama lain, berkumpul, berpisah, berkumpul semula dan mencari kebahagiaan dan penyelesaian yang benar di antara satu sama lain. Akhirnya, mereka dapat memulakan kehidupan normal! Rom-com dapat menjadi pelarian yang menyenangkan, tetapi kisah yang diceritakan terlalu sering, yang terlalu sering tidak mempunyai kepelbagaian, terlalu bergantung pada stereotaip jantina dan terlalu peduli dengan menjual jenama cinta yang mustahil untuk hidup: kajian tahun 2008 di Universiti Heriot Watt mendapati bahawa rom-com mempunyai kesan negatif terhadap hubungan, membuat kita mengejar standard cinta yang tidak dapat dicapai. Dalam proses menulis drama baru saya, Ross & Rachel, yang menghadapi mitos cinta moden dan dibuka di Edinburgh Festival Fringe pada bulan Ogos ini, saya harus banyak memikirkan percintaan dalam fiksyen dari Romeo dan Juliet hingga Pri

In [17]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()

In [18]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [19]:
refs = [filtered_right]
sys = filtered_left

In [20]:
sys[0]

'Anda tahu bagaimana kisah itu berlaku. Dua orang yang heteroseksual, hampir pasti orang kulit putih berjuang dalam hidup mereka. Mereka bertemu satu sama lain, berkumpul, berpisah, berkumpul semula dan mencari kebahagiaan dan penyelesaian yang benar di antara satu sama lain. Akhirnya, mereka dapat memulakan kehidupan normal! Rom-com dapat menjadi pelarian yang menyenangkan, tetapi kisah yang diceritakan terlalu sering, yang terlalu sering tidak mempunyai kepelbagaian, terlalu bergantung pada stereotaip jantina dan terlalu peduli dengan menjual jenama cinta yang mustahil untuk hidup: kajian tahun 2008 di Universiti Heriot Watt mendapati bahawa rom-com mempunyai kesan negatif terhadap hubungan, membuat kita mengejar standard cinta yang tidak dapat dicapai. Dalam proses menulis drama baru saya, Ross & Rachel, yang menghadapi mitos cinta moden dan dibuka di Edinburgh Festival Fringe pada bulan Ogos ini, saya harus banyak memikirkan percintaan dalam fiksyen dari Romeo dan Juliet hingga Pri

In [21]:
# for i in range(len(filtered_left)):
#     print(filtered_left[i])
#     print(filtered_right[i])
#     print(right[i])
#     print()

In [22]:
# filtered_right

In [23]:
# filtered_left

In [24]:
r = bleu.corpus_score(sys, refs)

In [25]:
r.__dict__

{'name': 'BLEU',
 'score': 62.34308405954152,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '82.6/67.5/57.2/49.3 (BP = 0.990 ratio = 0.990 hyp_len = 2604652 ref_len = 2630014)',
 'bp': 0.9903100596731002,
 'counts': [2152274, 1710097, 1409669, 1180379],
 'totals': [2604652, 2534909, 2465191, 2395664],
 'sys_len': 2604652,
 'ref_len': 2630014,
 'precisions': [82.63192165402518,
  67.46186943988917,
  57.18295255823991,
  49.271475465674655],
 'prec_str': '82.6/67.5/57.2/49.3',
 'ratio': 0.990356705325523}

In [26]:
tokenizer.push_to_hub('t5-tiny-finetuned-noisy-en-ms', organization='mesolitica')

In [27]:
model_gen.push_to_hub('t5-tiny-finetuned-noisy-en-ms', organization='mesolitica')

Upload file pytorch_model.bin:   0%|          | 4.00k/133M [00:00<?, ?B/s]

To https://huggingface.co/mesolitica/t5-tiny-finetuned-noisy-en-ms
   b05520d..9d4ca97  main -> main



'https://huggingface.co/mesolitica/t5-tiny-finetuned-noisy-en-ms/commit/9d4ca97c676624111999d4984253c994eda3f617'

In [28]:
model_tf.push_to_hub('t5-tiny-finetuned-noisy-en-ms', organization='mesolitica')

Upload file tf_model.h5:   0%|          | 4.00k/133M [00:00<?, ?B/s]

To https://huggingface.co/mesolitica/t5-tiny-finetuned-noisy-en-ms
   9d4ca97..3e28c7a  main -> main



'https://huggingface.co/mesolitica/t5-tiny-finetuned-noisy-en-ms/commit/3e28c7a2e41e8c4a8020065156ca534347ec723e'

In [29]:
!cd t5-tiny-finetuned-noisy-en-ms && git pull

Already up to date.


In [30]:
!cp t5-tiny-noisy-en-ms/*.tfevents.* t5-tiny-finetuned-noisy-en-ms
!cd t5-tiny-finetuned-noisy-en-ms && git add . && git commit -m 'add tensorboard' && git push

[main be0d9b5] add tensorboard
 1 file changed, 2 insertions(+), 2 deletions(-)
Uploading LFS objects: 100% (1/1), 61 MB | 6.4 MB/s, done.                      
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 16 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 374 bytes | 374.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
To https://huggingface.co/mesolitica/t5-tiny-finetuned-noisy-en-ms
   3e28c7a..be0d9b5  main -> main
